## Ejecutar el modelo

In [ ]:
import math
import numpy as np
from inspect import isfunction
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras import Sequential
from tensorflow import einsum
from keras import backend as K
from keras import metrics
from tensorflow.keras.models import save_model
from keras.losses import mse
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import preprocessing, Sequential
import time
import io
from tqdm import tqdm
import tensorflow_addons as tfa

# Tensorboard writer is created
tensorboard= tf.summary.create_file_writer(logdir='logs/{}'.format("Cars{}".format(int(time.time()))))

# Directory where the images for training the network are located is specified
directory="" # Format: "{directory}"

# Image size that the network should generate is specified
img_size = 128

# Folder where the trained model will be saved is specified
trained_models_folder ="" # Format: "{directory}\ "

# Folder where the images generated by the model will be saved is specified
generated_images_folder="" # Format: "{directory}\ "

# Batch size is specified
batch_size = 12

# Number of groups used in Group Normalization is specified
norm_groups = 4

# Number of timesteps for the forward and reverse process is set
timesteps=1000

# Filters to be applied in the network are set
filters = [32,64,128,256]

# Indicates which levels of the network use attention blocks
isAttentionLevel = [False, False, True, True]

# Number of residual blocks at each downsampling level (one more for upsampling)
nRes_blocks = 2

# Loads the car dataset, normalizes and horizontally flips its images
def get_loader(image_size):
    def augment(img):
        return tf.image.random_flip_left_right(img)
    def resize(image, height, width):
        resized_image = tf.image.resize(image, [height, width])
        return resized_image
    def train_preprocessing(x):
        img = tf.cast(x, dtype=tf.float32)
        img = tf.image.resize(img, size=(image_size, image_size), antialias=True)
        img = img / 127.5 - 1.0
        img = tf.clip_by_value(img, -1.0, 1.0)
        img = augment(img)
        return img
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        directory,
        label_mode=None,
        batch_size=None,
        shuffle=True,
        seed=123,
    )
    datasetmapeado = dataset.map(train_preprocessing, num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size, drop_remainder=True)
    return datasetmapeado

dataset=get_loader(img_size)

# Class containing the reverse and forward process
class DiffusionProcesses:
    def __init__(self, beta_start=1e-4, beta_end=0.02, timesteps=1000):
        # Beta vector of 1000 samples for the linear variance schedule is created
        self.beta_start = beta_start
        self.beta_end = beta_end
        self.timesteps = timesteps
        self.betas = betas = np.linspace(beta_start, beta_end, timesteps,dtype=np.float64)
        
        # Alpha vector for the linear variance schedule as 1-beta is created
        alphas = 1.0 - betas
        alphas_product = np.cumprod(alphas, axis=0)
        alphas_product_minus_one = np.append(1.0, alphas_product[:-1])
        self.betas = tf.constant(betas, dtype=tf.float32)
        
        # Product of alpha (t) vector is created
        self.alphas_product = tf.constant(alphas_product, dtype=tf.float32)
        # Product of alpha (t-1) vector for efficiency (so it's not calculated every time)
        self.alphas_product_minus_one = tf.constant(alphas_product_minus_one, dtype=tf.float32)

        # The following variables will contain all other operations needed for the forward and reverse process
        # (everything can be calculated from alpha and beta, but this increases efficiency and training speed)
        # Square root of the product of alpha (t) vector | sqrt(prod(α(t)))
        self.sqrt_alphas_product = tf.constant(np.sqrt(alphas_product), dtype=tf.float32)
        # Square root of 1 minus the product of alpha (t) vector | sqrt(1-prod(α(t)))
        self.sqrt_one_minus_alphas_product = tf.constant(np.sqrt(1.0 - alphas_product), dtype=tf.float32)
        # Square root of 1 divided by the product of alpha (t) vector | sqrt(1/prod(α(t)))
        self.sqrt_operation_alphas_product = tf.constant(np.sqrt(1.0 / alphas_product), dtype=tf.float32)
        # Square root of 1 divided by the product of alpha (t) minus 1 | sqrt(1/(prod(α(t)-1))
        self.sqrt_operation_alphas_minus_one_product = tf.constant(np.sqrt(1.0 / alphas_product - 1), dtype=tf.float32)

        # Natural logarithm of the variance of q as ln((β*(1-prod(α(t-1))))/(1-prod(α))) (log is used to later calculate the stddev in p_sample as e^(0.5*result(ln)))
        posterior_variance = (betas * (1.0 - alphas_product_minus_one)/(1.0 - alphas_product))
        self.posterior_variance = tf.constant(posterior_variance, dtype=tf.float32)
        self.posterior_log_variance = tf.constant(np.log(np.maximum(posterior_variance, 1e-20)), dtype=tf.float32)

        # To calculate the mean of q:
        # Mean of generated x as (sqrt(prod(α(t)))*(1-prod(α(t)))/(1-prod(α(t-1)))
        self.posterior_mean1 = tf.constant(np.sqrt(alphas)*(1.0 - alphas_product_minus_one)/(1.0 - alphas_product),dtype=tf.float32)
        # Mean of reconstructed x as (sqrt(prod(α(t-1)))*β/(1-prod(α))
        self.posterior_mean2 = tf.constant(np.sqrt(alphas_product_minus_one)*betas/(1.0 - alphas_product),dtype=tf.float32)
        
    # Function to correlate vectors with the current time epoch t 
    def _extract(self, a, t, x_shape):
        batch_size = x_shape[0]
        out = tf.gather(a, t)
        return tf.reshape(out, [batch_size, 1, 1, 1])

    # Function to add noise to the image (forward process)
    def forward_process(self, x_start, t, noise):
        x_start_shape = tf.shape(x_start)
        return self._extract(self.sqrt_alphas_product, t, tf.shape(x_start)) * x_start + self._extract(self.sqrt_one_minus_alphas_product, t, x_start_shape)* noise

    # Function to calculate the mean and log variance of q
    def q_posterior(self, x_recon, xt, t):
        xt_shape = tf.shape(xt)
        # Mean of q is calculated
        q_mean = self._extract(self.posterior_mean1, t, xt_shape) * xt + self._extract(self.posterior_mean2, t, xt_shape) * x_recon
        # Log variance of q is calculated
        q_log_variance = self._extract(self.posterior_log_variance, t, xt_shape)
        return q_mean, q_log_variance

    # Function to calculate reconstructed x
    def predict_start_from_noise(self, xt, t, noise):
        xt_shape = tf.shape(xt)
        return self._extract(self.sqrt_operation_alphas_product, t, xt_shape) * xt - self._extract(self.sqrt_operation_alphas_minus_one_product, t, xt_shape) * noise
    
    # Function to calculate the mean and log variance of p, which is that of q calculated above
    def p_mean_and_variance(self, pred_noise, x, t):
        # Reconstructed x is calculated 
        x_recon = self.predict_start_from_noise(x, t=t, noise=pred_noise)
        x_recon = tf.clip_by_value(x_recon, -1.0, 1.0)
        # q_posterior is called
        posterior_mean, posterior_log_variance = self.q_posterior(x_recon=x_recon, xt=x, t=t)
        return posterior_mean, posterior_log_variance

    # Function to remove noise from the image (reverse process)
    def p_sample(self, pred_noise, x, t):
        # p_mean_and_variance is called
        posterior_mean, posterior_log_variance = self.p_mean_and_variance(pred_noise, x=x, t=t)
        # Noise vector z is created
        noise = tf.random.normal(shape=x.shape, dtype=x.dtype)
        # If t is 0, only the mean is returned
        zero = tf.reshape(1 - tf.cast(tf.equal(t, 0), tf.float32), [tf.shape(x)[0], 1, 1, 1])
        # Returns μt(xgen,xrec)+σt*z
        return posterior_mean + zero * tf.exp(0.5 * posterior_log_variance) * noise

# An object of the DiffusionProcesses class is created
utils = DiffusionProcesses(timesteps=1000)

# Function to initialize the Kernel
def kernel_init(scale):
    scale = max(scale, 1e-10)
    return keras.initializers.VarianceScaling(scale, mode="fan_avg", distribution="uniform")

# Sinusoidal positional embedding class for the Transformer
class TimeEmbedding(layers.Layer):
    def __init__(self, dim, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim
        self.half_dim = dim // 2
        self.emb = math.log(10000) / (self.half_dim - 1)
        self.emb = tf.exp(tf.range(self.half_dim, dtype=tf.float32) * -self.emb)

    def call(self, inputs):
        inputs = tf.cast(inputs, dtype=tf.float32)
        emb = inputs[:, None] * self.emb[None, :]
        emb = tf.concat([tf.sin(emb), tf.cos(emb)], axis=-1)
        return emb

# Function that introduces the timeEmbedding through an MLP 
def TimeMLP(units, activation_fn=keras.activations.swish):
    def apply(inputs):
        # First MLP layer
        timeEmbedding = layers.Dense(units, activation=activation_fn, kernel_initializer=kernel_init(1.0))(inputs)
        # Second MLP layer
        timeEmbedding = layers.Dense(units, kernel_initializer=kernel_init(1.0))(timeEmbedding)
        return timeEmbedding
    return apply

# Attention block class
class AttentionBlock(layers.Layer):
    def __init__(self, units, groups=8, **kwargs):
        self.units = units
        self.groups = groups
        super().__init__(**kwargs)
        # Initializes the GroupNormalization layer
        self.norm = tfa.layers.GroupNormalization(groups=groups)
        # Initializes the Dense layer for the query
        self.query = layers.Dense(units, kernel_initializer=kernel_init(1.0))
        # Initializes the Dense layer for the key
        self.key = layers.Dense(units, kernel_initializer=kernel_init(1.0))
        # Initializes the Dense layer for the value
        self.value = layers.Dense(units, kernel_initializer=kernel_init(1.0))
        # Initializes the Dense layer for projection
        self.proj = layers.Dense(units, kernel_initializer=kernel_init(0.0))

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        height = tf.shape(inputs)[1]
        width = tf.shape(inputs)[2]
        # Scale is set as 1/sqrt(num_channels)
        scale = tf.cast(self.units, tf.float32) ** (-0.5)

        # Inputs are normalized
        inputs = self.norm(inputs)
        q = self.query(inputs)
        k = self.key(inputs)
        v = self.value(inputs)

        # Mathmul*Scale is performed
        attn_score = tf.einsum("bhwc, bHWc->bhwHW", q, k) * scale
        # Similarity is calculated
        attn_score = tf.reshape(attn_score, [batch_size, height, width, height * width])
        
        # Softmax is calculated
        attn_score = tf.nn.softmax(attn_score, -1)
        attn_score = tf.reshape(attn_score, [batch_size, height, width, height, width])

        # Final Mathmul is performed
        proj = tf.einsum("bhwHW,bHWc->bhwc", attn_score, v)
        proj = self.proj(proj)#Projection
        return inputs + proj

# Residual block function
def ResidualBlock(width, groups=8, activation_fn=keras.activations.swish):
    def apply(inputs):
        x, t = inputs
        input_width = x.shape[3]

        # Checks if the input vector dimension matches the output, if so adds input to output
        if input_width == width:
            add = x
        else: # Otherwise, adds input to output after a convolution
            add = layers.Conv2D(width, kernel_size=1, kernel_initializer=kernel_init(1.0))(x)

        timeEmbedding = activation_fn(t)
        # Dense layer used for timeEmbedding
        timeEmbedding = layers.Dense(width, kernel_initializer=kernel_init(1.0))(timeEmbedding)[:, None, None, :]
        # Group Normalization before the first convolutional layer
        x = tfa.layers.GroupNormalization(groups=groups)(x)
        x = activation_fn(x)
        # First convolution
        x = layers.Conv2D(width, kernel_size=3, padding="same", kernel_initializer=kernel_init(1.0))(x)

        x = layers.Add()([x, timeEmbedding])
        # Group Normalization before the second convolutional layer
        x = tfa.layers.GroupNormalization(groups=groups)(x)#GroupNormalization right before the second convolutional layer
        x = activation_fn(x)
        
        # Second convolutional layer that receives the timeEmbedding
        x = layers.Conv2D(width, kernel_size=3, padding="same", kernel_initializer=kernel_init(0.0))(x)
        x = layers.Add()([x, add])
        return x
    return apply

# DownBlock
def DownBlock(width):
    def apply(x):
        x = layers.Conv2D(width,kernel_size=3,strides=2,padding="same",kernel_initializer=kernel_init(1.0))(x)
        return x
    return apply


def UpBlock(width, interpolation="nearest"):
    def apply(x):
        x = layers.UpSampling2D(size=2, interpolation=interpolation)(x)
        x = layers.Conv2D(width, kernel_size=3, padding="same", kernel_initializer=kernel_init(1.0))(x)
        return x

    return apply

# Modified U-Net function
def build_model(img_size,img_channels,filters,isAttentionLevel,nRes_blocks=2,norm_groups=8,interpolation="nearest",activation_fn=keras.activations.swish):
    # Input layer for image dimensions
    image_input = layers.Input(shape=(img_size, img_size, img_channels), name="image_input")
    # Input layer for timeEmbedding dimensions
    time_input = keras.Input(shape=(), dtype=tf.int64, name="time_input")
    
    # First convolutional layer to set the number of input channels for the modified U-Net
    x = layers.Conv2D(filters[0],kernel_size=(3, 3),padding="same",kernel_initializer=kernel_init(1.0))(image_input)
    timeEmbedding = TimeEmbedding(dim=filters[0] * 4)(time_input)#Creates the Transformer sinusoidal position embedding
    timeEmbedding = TimeMLP(units=filters[0] * 4, activation_fn=activation_fn)(timeEmbedding)#Creates the TimeEmbedding

    # Initializes the skip connections vector
    skips = [x]
    # Downsampling blocks loop. As many as there are filters
    for i in range(len(filters)):
        for _ in range(nRes_blocks):
            x = ResidualBlock(filters[i], groups=norm_groups, activation_fn=activation_fn)([x, timeEmbedding])
            if isAttentionLevel[i]:
                x = AttentionBlock(filters[i], groups=norm_groups)(x)
            skips.append(x)
        if filters[i] != filters[-1]:
            x = DownBlock(filters[i])(x)#DownBlock Blocks
            skips.append(x)
    
    # Middle block
    x = ResidualBlock(filters[-1], groups=norm_groups, activation_fn=activation_fn)([x, timeEmbedding])
    x = AttentionBlock(filters[-1], groups=norm_groups)(x)
    x = ResidualBlock(filters[-1], groups=norm_groups, activation_fn=activation_fn)([x, timeEmbedding])
    
    # Upsampling blocks loop. As many as there are filters
    for i in reversed(range(len(filters))):
        for _ in range(nRes_blocks+1):
            # Concatenates the output of the parallel downsampling parts with the inputs of the upsampling parts via skip connections
            x = layers.Concatenate(axis=-1)([x, skips.pop()])
            x = ResidualBlock(filters[i], groups=norm_groups, activation_fn=activation_fn)([x, timeEmbedding])
            if isAttentionLevel[i]:
                x = AttentionBlock(filters[i], groups=norm_groups)(x)
        if i != 0:
            x = UpBlock(filters[i], interpolation=interpolation)(x)

    # Final block
    # Output is normalized with Group Normalization
    x = tfa.layers.GroupNormalization(groups=norm_groups)(x)
    x = activation_fn(x)
    # Output channels are reduced to three dimensions (RGB)
    x = layers.Conv2D(3, (3, 3), padding="same", kernel_initializer=kernel_init(0.0))(x)
    return keras.Model([image_input, time_input], x, name="U-Net_Modified")

# Variable to hold the model is created
network = build_model(img_size=img_size,img_channels=3,filters=filters,isAttentionLevel=isAttentionLevel,nRes_blocks=nRes_blocks,norm_groups=norm_groups,activation_fn=keras.activations.swish)

# Model is compiled specifying its optimizer and objective function (MSE)
optimizer=keras.optimizers.Adam(learning_rate=2e-4)
network.compile(loss=keras.losses.MeanSquaredError(),optimizer=optimizer)

# Returns the summary of the DDPM
network.summary()

# DDPM loss function
def ddpm_loss(noise, pred_noise):
    diffusion_loss = tf.keras.losses.MeanSquaredError()(noise, pred_noise)
    return diffusion_loss

# Returns images in a format that allows storing them in Tensorboard
def plot_to_image(figure):
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(figure)
    buf.seek(0)
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    image = tf.expand_dims(image, 0)
    return image

# Creates a 5x5 grid with the received images
def image_grid(images):
    figure = plt.figure(figsize=(10,10))
    for i in range(images.shape[0]):
        img = preprocessing.image.array_to_img((images[i] + 1 / 2))
        plt.subplot(5, 5, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(img)
    return figure

# Function to generate images using the reverse process
def generate_images(seed, num_images=25):
    for t in reversed(range(0, timesteps)):
        print(f"\rStage t = {t}", end=" ")
        current_t = tf.cast(tf.fill(num_images, t), dtype=tf.int64)
        # Predicted noise is calculated
        pred_noise = network.predict([seed, current_t], verbose=0, batch_size=num_images)
        # Images are generated by performing the reverse process
        seed = utils.p_sample(pred_noise, seed, current_t)
    return seed

# Function that calls generate_images with samples and saves the generated images in the specified folder
def plot_images(folder, seed, epoch=None, logs=None, num_rows=5, num_cols=5, figsize=(5, 5)):
    generated_images = generate_images(seed, num_images=num_rows * num_cols)
    generated_images = (
            tf.clip_by_value(generated_images * 127.5 + 127.5, 0.0, 255.0)
            .numpy()
            .astype(np.uint8)
        )
    _, ax = plt.subplots(num_rows, num_cols, figsize=figsize)
    for i, image in enumerate(generated_images):
        if num_rows == 1:
            ax[i].imshow(image)
            ax[i].axis("off")
        else:
            ax[i // num_cols, i % num_cols].imshow(image)
            ax[i // num_cols, i % num_cols].axis("off")

    plt.tight_layout()
    plt.savefig(folder + 'generated_image_epoch_%d.png' % epoch)
    plt.close()
    return generated_images

# Seed is created to always generate the same images and thus compare the network's evolution
tf.random.set_seed(347)
seed = tf.random.normal(shape=(25, img_size, img_size, 3), dtype=tf.float32)

# Network is trained via train_step
@tf.function
def train_step(images):
    # Uniform random time vector of 1000 samples is generated
    t = tf.random.uniform(minval=0, maxval=timesteps, shape=(batch_size,), dtype=tf.int64)

    # Network is trained
    with tf.GradientTape() as tape:
        # Noise vector to be added to the images is created
        noise = tf.random.normal(shape=tf.shape(images), dtype=images.dtype)
        # Forward process is called to apply the noise vector to the images
        images_t = utils.forward_process(images, t, noise)
        # Predicted noise is extracted from the network
        pred_noise = network([images_t, t], training=True)
        # Network error is calculated
        diffusion_loss = ddpm_loss(noise, pred_noise)

    # Gradients of the discriminator are updated
    gradients = tape.gradient(diffusion_loss, network.trainable_weights)
    optimizer.apply_gradients(zip(gradients, network.trainable_weights))

    # Returns the sum of the network error
    return diffusion_loss

# Network training function
def train(epochs, model):
    generated_images=plot_images(generated_images_folder, seed, 0)
    for epoch in range(epochs):
        print('Current training epoch {} (out of {}).'.format(epoch+1, epochs))
        # Loop that iterates over the dataset to train the network
        for image_batch in tqdm(dataset):
            diffusion_loss=train_step(image_batch)
        # Model error is printed in Tensorboard
        with tensorboard.as_default():
            tf.summary.scalar('Loss Diffusion', diffusion_loss.numpy(), step=epoch)
        if epoch % 10 == 0:
            generated_images=plot_images(generated_images_folder, seed, epoch)
            fig=image_grid(generated_images[:25])
            # Images generated by the model are printed in Tensorboard
            with tensorboard.as_default():
                tf.summary.image('Generated images', plot_to_image(fig), step=epoch)
            # Model weights are saved
            network.save_weights(trained_models_folder + "Diffusion_epoch_%d" % epoch)
    # In the last iteration, the model and the last produced images are saved
    plot_images(generated_images_folder, seed, epoch)
    network.save_weights(trained_models_folder + "Diffusion_epoch_%d" % epoch)

# Calls the train function to start training
train(150,network)

## Cargar modelo

In [ ]:
# Specify the folder where the trained model will be loaded from (the model must be created before loading, so you could temporarily comment out line 466 of the previous code)
epoch = 0  # Specify the epoch of the model you want to load
network.load_weights(trained_models_folder + "Diffusion_epoch_%d" % epoch)